In [ ]:
import os
from time import sleep

# Functional Programming in Python

## What to do
* Write functions, not classes
* Avoid "inplace" data manipulation and side effects
* Use immutable data structures / treat data structures as immutable
* To combine and configure functionality, pass functions as parameters

## Benefits
&rarr; Keeps code repeatable: Same function with same input = same output

&rarr; Better reusability

&rarr; Easier debugging

&rarr; Works great with REPL- or Jupyter-driven development

&rarr; Parallelising is easy if there is no shared state!

&rarr; No more messy inheritance

&rarr; More straight-forward serialisation

# Intro

Often, we create classes as a storage for a configuration before execution:

In [ ]:
class HelloSayer:
    def __init__(self, name: str) -> None:
        self.name = name
        
    def say_hello(self):
        print(f'Hello {self.name}')

To execute the function, we need to first initialise an instance of the class and then call its method:

In [ ]:
obj = HelloSayer('EuroPython')
obj.say_hello()

This is fairly verbose, considering that all we really need is:

In [ ]:
def say_hello(name: str):
    print(f'Hello {name}')

Which we can then simply call:

In [ ]:
say_hello('better code!')

*Note*: Strictly speaking, the above is not a pure function, as `print` is a side effect. We'll get better later on.

## Builtins: `functools`
What if you would like to configure a function before using it? There are actually tools to do that, e.g. `functools.partial`:

In [ ]:
from functools import partial

say_hello_to_functools = partial(say_hello, name='functools')

In [ ]:
say_hello_to_functools()

Remember that you can pass functions as arguments, too. So, now you can combine functions:

In [ ]:
def do_a_few_things(*functions):
    """
    Calls all functions passed (without any arguments)
    """
    for function in functions:
        function()

In [ ]:
do_a_few_things(
    say_hello_to_functools,
    partial(say_hello, name='EuroPython'),
    partial(print, "Isn't this great?")
)

Now, as mentioned before, we don't really want side effects, which includes use of the `print` statement. So let's re-write our function without:

In [ ]:
def greeting(name: str):
    return f'Hello {name}'

Because we don't have the side effect any more, it does actually not matter anymore when and where we execute the function, i.e. it's easier to parallelise, and easier to combine with other functionality.

But we do want to print the results. But how? Easy:

In [ ]:
def print_all(*args):
    [print(arg) for arg in args]

print_all(
    greeting('EuroPython'),
    greeting('Everyone'),
    greeting('Beautiful code'),
)

## Builtins: list comprehensions, `map`, `filter`, `reduce`

Quite often we want to apply a function or calculation to each item in an existing list, ending up with constructs like this:

In [ ]:
inputs = range(10)

outputs = []
for x in inputs:
    result = (7 * x) ** 3
    outputs.append(result)
    
outputs

This is not only verbose. It is also computationally expensive as you're explicitly appending to a list. It is also very prone to errors (e.g. using it multiple times without resetting `outputs`. 

It can be solved much more elegantly with a list comprehension:

In [ ]:
[(7 * x) ** 3 for x in inputs]

... or you can "map" your inputs over a (lambda) function:

In [ ]:
list(map(lambda x: (7 * x) ** 3, inputs))

*Note*: `map` is executed "lazily", i.e. it returns a generator. By calling `list()`, we force eager execution and collect all the values into a python list.

You can separate this further, which is especially useful for more complex functions to be applied to each element in the list:

In [ ]:
def my_calculation(x):
    """
    Returns my calculation for one input.
    Having this as a separate function separates concerns and makes it very usable!
    """
    return (7 * x) ** 3

In [ ]:
list(map(my_calculation, inputs))

The functions `filter` and `reduce` work very similarly:

In [ ]:
list(filter(lambda x: x < 10000, outputs))

In [ ]:
from functools import reduce

reduce(lambda x, y: (x + y) / 2, outputs)

# Picking up the pace...
Up to here, this was an introduction to some basic functionality. Let's jump in a bit deeper and try out some libraries.

## Easy multithreading with `pmap`
This is essentially the `map` function from earlier, but parallelised automatically!

In [ ]:
!pip install python-pmap

In [ ]:
def my_slow_function(x):
    sleep(1)  # artificially make this function slow
    return x**2

In [ ]:
%%time
[my_slow_function(x) for x in range(10)]

In [ ]:
%%time
list(map(my_slow_function, range(10)))

In [ ]:
%%time
from pmap import pmap

list(pmap(my_slow_function, range(10)))

## `toolz`
The [toolz](https://toolz.readthedocs.io/en/latest/api.html) library is incredibly useful for functional programming in python. Here are just a few examples:

In [ ]:
from toolz import *

d = {
    'some': 123,
    'values': 42,
    'in': 1,
    'a': 96,
    'dictionary': 17
}

In [ ]:
# apply map to keys only
keymap('{}_post'.format, d)

In [ ]:
# apply map to values only
valmap((100).__mul__, d)

*Note*: In `python-pmap`, there are parallelised versions for these as well, called `pkeymap` and `pvalmap`

In [ ]:
def iseven(x): 
    return x % 2 == 0

groupby(iseven, [1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
d1 = {'k1': 1, 'k2': 2}
d2 = {'k1': 4,          'k3': 3}

In [ ]:
merge(d1, d2)  # meres dictionaries (latest overwrites)

In [ ]:
merge_with(sum, d1, d2)  # merges by summing values up

There are lots and lots of useful functions there, just try them out! One of my favourites is `thread_last` - although I won't show that here, and instead will show `coconut`'s piping later

# Pandas
Yes, pandas can be used in a functional way, mainly by not changing dataframes inplace, and by not assigning columns directly.

In [ ]:
import pandas as pd

df = pd.DataFrame(pd.np.random.random((5,3)), columns=list('abc'))
df

Instead of something like:

```python
df['sum'] = df.sum(axis=1)
```

use `df.assign`:

In [ ]:
df.assign(sum=df.sum(axis=1))

Note how this did *not* change the original dataframe, i.e. if you jump around cells in your notebook, outputs will not change:

In [ ]:
df

You can chain these expressions, use lambdas; and of course you can assign them to a new variable:

In [ ]:
df_transformed = (df
    .assign(sum=lambda df: df.sum(axis=1))
    .assign(a_percent=lambda df: df['a'] / df['sum'])
    .drop(index=[1,3]))

df_transformed

Doing this consistently will make your data transformations more reliable and repeatable.

# `functools`: `singledispatch`
You may have asked yourself how to "translate" object oriented constructs such as inheritance and method overloading to the functional world. A very powerful tool to do this is "dispatching" to different implementations of a function depending on its inputs.

Python comes with a simple construct to do this: `singledispatch`. It can dispatch only on the type of the first argument to a function:

In [ ]:
from functools import singledispatch

In [ ]:
@singledispatch
def negate(x):
    """
    negate a value, default implementation (returns -x)
    """
    return -x

In [ ]:
negate(5)

In [ ]:
negate('hello')

In [ ]:
@negate.register(str)
def negate_str(x):
    return f'the opposite of {x}'

In [ ]:
negate('hello')

In [ ]:
negate(5)

## `multimethodic` (multi-dispatch)
A more powerful dispatch mechanism is implemented in the `multimethodic` package. As it is more flexible, you will have to define its dispatcher:

In [ ]:
from multimethodic import MultiMethod, Default

In [ ]:
def dispatch_by_two_types(x, y):
    return type(x), type(y)

multiply_or_repeat = MultiMethod('multiply_or_repeat', dispatch_by_two_types)

In [ ]:
@multiply_or_repeat.method(Default)
def multiply_or_repeat_default(x, y):
    return x * y

In [ ]:
@multiply_or_repeat.method((str, int))
def multiply_or_repeat_with_str(x, y):
    return ', '.join([x for _ in range(y)])

In [ ]:
multiply_or_repeat(6, 3)

In [ ]:
multiply_or_repeat('Hello', 3)

# Coconut
[Coconut](http://coconut-lang.org/) is a functional language extension to python and has some very concise, functional constructs. Although the language support in common IDEs is not great yet, it can be a very neat, clean and fast way to write functional, beautiful, repeatable and parallelisable code.

Installation is easy:

In [ ]:
!pip install coconut
!coconut --jupyter

In [ ]:
%load_ext coconut

You can either change the notebook kernel to coconut, or you can use the `%%coconut` magic to mark a jupyter cell as coconut

## Simple Example
Let's look at this fairly simple piece of functional python:

In [ ]:
inputs = range(1,30)

def my_function(x):
    return x ** 3 % 1000

mapped = list(map(my_function, inputs))
filtered = list(filter(lambda x: x < 100, mapped))

filtered

To make this more concise, we could nest the functions:

In [ ]:
list(filter(lambda x: x < 100, map(my_function, inputs)))

That is more concise, but it becomes trickier to read -- in particular regarding the order of execution. Please don't do this in your, it's only to illustrate how Coconut can help write cleaner, more concise code! 

As a first improvement, we can use a pipe operator `|>`, taking the input on the left and, one step at a time, applying functions in order of reading: from left to right.

In [ ]:
%%coconut
inputs |> partial(map, my_function) |> partial(filter, lambda x: x < 100) |> list

Still not perfect yet... Really, those calls to `partial` are not nice to read. Coconut a trick for that as well: A `$` between a function name and its opening bracket makes that call a `partial` assignment. Using that we get:

In [ ]:
%%coconut
inputs |> map$(my_function) |> filter$(lambda x: x < 100) |> list

And, using coconut's arrow syntax for functions, parallelising with `pmap`, and a bit of formatting:

In [ ]:
%%coconut
(inputs 
 |> pmap$(my_function) 
 |> filter$(x -> x < 100) 
 |> list)

Really, this is just the tip of the iceberg. If you like the concept, read more on the coconut website [coconut-lang.org](http://coconut-lang.org/)